# WeatherPy 

In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import json 
import requests
import time
import scipy.stats as sts
from scipy import stats
import openweathermapy.core as owm
import csv 

#Import API Key
from config import weather_api_key

# to get the city latitude and longtiude 
from citipy import citipy

# Range of latitudes and longitudes
#https://desktop.arcgis.com/en/arcmap/10.3/guide-books/map-projections/about-geographic-coordinate-systems.htm
lat_range = (-90, 90)
lng_range = (-180, 180)

# Cities Summary 

In [8]:
#Set up lists to hold reponse info for latitude and lontitude
lat_lngs = []
cities = []
countries = []

#Get the random lat and lng then zip together
#https://stackoverflow.com/questions/55230785/how-can-i-generate-a-seed-for-np-random-uniform
lats = np.random.uniform(low=-90.00, high=90.00, size=1500)
lngs = np.random.uniform(low=-180.00, high=180.00, size=1500)
lat_lngs = zip(lats, lngs)


#Get the nearest city based on latitude and lontitude
#https://github.com/wingchen/citipy/blob/master/tests.py
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    country = citipy.nearest_city(lat_lng[0], lat_lng[1]).country_code
    if city not in cities:
        cities.append(city)
        countries.append(country)

#Create DataFrame for city and country         
City_latlong_data = pd.DataFrame({"City": cities, "Country": countries})
City_latlong_data = City_latlong_data.reset_index(drop=True)
City_latlong_data.head() 

#set up additional columns to hold information
City_latlong_data["Cloudiness"] = ""
City_latlong_data["Humidity"] = ""
City_latlong_data["Date"] = ""
City_latlong_data["Max Temp"] = ""
City_latlong_data["Wind Speed"] = ""

#Print the city count
len(cities)

625

In [9]:
#Create a dataframe to store all the latitude and longitude
Cities_df = pd.DataFrame()
Cities_df = Cities_df.append(pd.DataFrame.from_dict({ "Latitude": lats, "Longitude": lngs}))        
Cities_df

#Merge two dataframes to display all the additional columns, drop duplicates and drop nan vaulues
City_Data = [City_latlong_data, Cities_df]        
City_Data_DF = pd.concat([City_latlong_data, Cities_df], axis=1)
City_Data_DF = City_Data_DF.drop_duplicates()
City_Data_DF = City_Data_DF.dropna()
City_Data_DF = City_Data_DF.reset_index(drop=True)
City_Data_DF = pd.DataFrame(City_Data_DF)
City_Data_DF.head(20)

,City,Country,Cloudiness,Humidity,Date,Max Temp,Wind Speed,Latitude,Longitude
0,hobart,au,,,,,,-72.889249,143.382685
1,rikitea,pf,,,,,,-60.388120,-133.390419
2,ambulu,id,,,,,,-12.310450,112.842907
3,igis,ch,,,,,,46.970568,9.641243
4,cape town,za,,,,,,-56.759346,-2.754619
5,dikson,ru,,,,,,82.871498,72.503527
6,ellensburg,us,,,,,,47.384877,-120.900448
7,belushya guba,ru,,,,,,87.457357,56.364219
8,hermanus,za,,,,,,-69.533862,-1.365960
9,devils lake,us,,,,,,48.455021,-98.875620


In [10]:
##* Randomly select **at least** 500 unique (non-repeat) cities based on latitude and longitude.
random_selected_cities = City_Data_DF.sample(500)
#Print the selected cities Dataframe
random_selected_cities = random_selected_cities.reset_index(drop=True)
random_selected_cities = pd.DataFrame(random_selected_cities)
random_selected_cities

,City,Country,Cloudiness,Humidity,Date,Max Temp,Wind Speed,Latitude,Longitude
0,grand-santi,gf,,,,,,39.033504,122.137056
1,thiruvananthapuram,in,,,,,,21.977548,-104.420110
2,udachnyy,ru,,,,,,47.922191,37.577266
3,vaini,to,,,,,,-26.942396,172.679406
4,canchungo,gw,,,,,,-14.520698,73.500881
...,...,...,...,...,...,...,...,...,...
495,palm bay,us,,,,,,59.290278,-21.084677
496,brockville,ca,,,,,,-22.247956,-115.948180
497,biak,id,,,,,,7.131549,54.426850
498,mahadday weyne,so,,,,,,-5.006289,-88.466439


# Get API Calls¶

In [14]:
#https://gt.bootcampcontent.com/GT-Coding-Boot-Camp/gt-atl-data-pt-03-2020-u-c/blob/master/06-Python-APIs/3/Activities/04-Ins_NearestRestr/Solved/NearestRestr.ipynb
#URL for get requests to retrieve weather data - 
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)
     
# set up lists to hold reponse info
city_name =[]
lat = []
lon = []
temp =[]
temperature_max = []
date = []
humidity = []
wind_speed = []
country = []
cloudiness = []

#counter for data set number of calls
record_count = 1

# Loop through the list of cities and perform a request
#Used Activity 06-Python-APIs/2/Activities/10-Stu_API_Exceptions/Solved/api_exceptions.ipynb
print(f"Generating Data Retrival")

for city in cities:
    
# Try to grab the lists of city responses if availble from json response
    try:
        response = requests.get(f"{url}&q={city}").json()
        #Making sure my url printed 
#        print(response)
        city_name.append(response['name'])
        lat.append(response['coord']['lat'])
        lon.append(response['coord']['lon'])
        temperature_max.append(response['main']['temp_max'])
        date.append(response['dt'])
        humidity.append(response['main']['humidity'])
        wind_speed.append(response['wind']['speed'])
        country.append(response['sys']['country'])
        cloudiness.append(response['clouds']['all'])
        city = response['name']
        print(f"The weather API responded with:# {record_count} | {city}")
        record_count = record_count + 1

        # API CALLS TO SLEEP FOR 1 SECONDS DUE TO FREE 60 CALLS/MIN 
        time.sleep(1)
        
    
    except KeyError: 
        print("City missing...Skipping")
        
print("------------------------")
print("Generated Data Completed")
print("------------------------")

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=9246be5369046e9d6295393ab5911965
Generating Data Retrival
The weather API responded with:# 1 | Hobart
The weather API responded with:# 2 | Rikitea
The weather API responded with:# 3 | Ambulu
The weather API responded with:# 4 | Igis
The weather API responded with:# 5 | Cape Town
The weather API responded with:# 6 | Dikson
The weather API responded with:# 7 | Ellensburg
City missing...Skipping
The weather API responded with:# 8 | Hermanus
The weather API responded with:# 9 | Devils Lake
The weather API responded with:# 10 | Ushuaia
The weather API responded with:# 11 | Ballina
The weather API responded with:# 12 | Ribeira Grande
City missing...Skipping
City missing...Skipping
The weather API responded with:# 13 | Maun
The weather API responded with:# 14 | Enumclaw
The weather API responded with:# 15 | Pevek
The weather API responded with:# 16 | Dhrāngadhra
The weather API responded with:# 17 | Nikolskoye
The weather API

In [17]:
#checking the counts and making sure the information is there needed
# ERROR RECEIVED ABOUT valueerror: arrays must all be same length
print(len(cities), len(country), len(cloudiness), len(humidity), len(date), len(temperature_max),
     len(wind_speed), len(lat), len(lon))

625 577 577 577 577 577 577 577 577


# RAW DATA 

In [91]:
city_weatherpy_dict = {"City": cities, "Country": country, 
                       "Cloudiness %": cloudiness, "Humidity": humidity,
                       "Date": date, "Max Temp (F)": temperature_max,
                       "Wind Speed (mph)": wind_speed, "Latitude": lat,
                       "Longitude": lon
                      }
city_weatherpy_data = pd.DataFrame.from_dict(city_weatherpy_dict, orient='index')
city_weatherpy_data.transpose()

,City,Country,Cloudiness %,Humidity,Date,Max Temp (F),Wind Speed (mph),Latitude,Longitude
0,hobart,AU,75,81,1587333184,51.8,6.93,-42.88,147.33
1,rikitea,PF,100,68,1587333375,76.87,6.96,-23.12,-134.97
2,ambulu,ID,100,76,1587333225,78.46,2.91,-8.35,113.61
3,igis,CH,100,60,1587333227,54.59,3.33,46.95,9.57
4,cape town,ZA,75,77,1587333059,64,17.22,-33.93,18.42
...,...,...,...,...,...,...,...,...,...
620,ca mau,None,None,None,None,None,None,None,None
621,wageningen,None,None,None,None,None,None,None,None
622,karoi,None,None,None,None,None,None,None,None
623,port hedland,None,None,None,None,None,None,None,None


In [104]:
transposed_data = city_weatherpy_data.T
transposed_data

,City,Country,Cloudiness %,Humidity,Date,Max Temp (F),Wind Speed (mph),Latitude,Longitude
0,hobart,AU,75,81,1587333184,51.8,6.93,-42.88,147.33
1,rikitea,PF,100,68,1587333375,76.87,6.96,-23.12,-134.97
2,ambulu,ID,100,76,1587333225,78.46,2.91,-8.35,113.61
3,igis,CH,100,60,1587333227,54.59,3.33,46.95,9.57
4,cape town,ZA,75,77,1587333059,64,17.22,-33.93,18.42
...,...,...,...,...,...,...,...,...,...
620,ca mau,None,None,None,None,None,None,None,None
621,wageningen,None,None,None,None,None,None,None,None
622,karoi,None,None,None,None,None,None,None,None
623,port hedland,None,None,None,None,None,None,None,None


In [110]:
transposed_data.dropna(inplace=True)
transposed_data

,City,Country,Cloudiness %,Humidity,Date,Max Temp (F),Wind Speed (mph),Latitude,Longitude
0,hobart,AU,75,81,1587333184,51.8,6.93,-42.88,147.33
1,rikitea,PF,100,68,1587333375,76.87,6.96,-23.12,-134.97
2,ambulu,ID,100,76,1587333225,78.46,2.91,-8.35,113.61
3,igis,CH,100,60,1587333227,54.59,3.33,46.95,9.57
4,cape town,ZA,75,77,1587333059,64,17.22,-33.93,18.42
...,...,...,...,...,...,...,...,...,...
572,komsomolskiy,VN,0,81,1587334020,78.53,3.69,9.18,105.15
573,maua,NL,0,40,1587334021,54,12.75,51.97,5.67
574,tianpeng,ZW,65,62,1587334022,62.91,4.92,-16.81,29.69
575,maldonado,AU,74,94,1587334023,75.2,3.36,-20.32,118.57


In [112]:
#Output for my csv file
transposed_data.to_csv("./output_data/what_weatherpy_data.csv") 

In [113]:
#Read my Output file to make sure data is formatted correctly for my csv file
transposed_data = pd.read_csv("./output_data/what_weatherpy_data.csv")
transposed_data.head()

,Unnamed: 0,City,Country,Cloudiness %,Humidity,Date,Max Temp (F),Wind Speed (mph),Latitude,Longitude
0,0,hobart,AU,75,81,1587333184,51.80,6.93,-42.88,147.33
1,1,rikitea,PF,100,68,1587333375,76.87,6.96,-23.12,-134.97
2,2,ambulu,ID,100,76,1587333225,78.46,2.91,-8.35,113.61
3,3,igis,CH,100,60,1587333227,54.59,3.33,46.95,9.57
4,4,cape town,ZA,75,77,1587333059,64.00,17.22,-33.93,18.42


In [137]:
#Plotting the Data - Latitude VS Temperature (F) plot 
x_values =
y_values = 
plt.scatter(x_values, y_values)
plt.grid()
plt.xlabel()
plt.ylabel()
plt.title(f"Latitude VS Temperature")
#plt.savefig("./output_data/")
#Output_file = "./output_data/What_Weather_Output_Data/cities.csv"
plt.show()

SyntaxError: invalid syntax (<ipython-input-137-e0e06fd27664>, line 2)

In [ ]:
#After each plot add a sentence or too explaining what the code is and analyzing.


In [ ]:
#Create a scatte plot for Humidity (%) vs. Latitude
x_values =
y_values = 
plt.scatter(x_values, y_values)
plt.grid()
plt.xlabel()
plt.ylabel()
plt.title(f"Latitude VS Humidity")
#plt.savefig("./output_data/")
#Output_file = "./output_data/What_Weather_Output_Data/cities.csv"
plt.show()

In [ ]:
#After each plot add a sentence or too explaining what the code is and analyzing.


In [ ]:
#Create a scatter plot for Cloudiness (%) vs. Latitude
x_values =
y_values = 
plt.scatter(x_values, y_values)
plt.grid()
plt.xlabel()
plt.ylabel()
plt.title(f"Latitude VS Cloudiness")
#plt.savefig("./output_data/")
#Output_file = "./output_data/What_Weather_Output_Data/cities.csv"
plt.show()

In [ ]:
#After each plot add a sentence or too explaining what the code is and analyzing.

In [ ]:
#Create a scatter plot for Wind Speed (mph)) vs. Latitude
x_values =
y_values = 
plt.scatter(x_values, y_values)
plt.grid()
plt.xlabel()
plt.ylabel()
plt.title(f"Latitude VS Wind Speed")
#plt.savefig("./output_data/")
#Output_file = "./output_data/What_Weather_Output_Data/cities.csv"
plt.show()

In [ ]:
#After each plot add a sentence or too explaining what the code is and analyzing.

In [ ]:
#Create a linear regression plot 

In [ ]:
# Create the DataFrames for the hemispheres 

#Your second requirement is to run linear regression on each relationship, only this time separating them into Northern Hemisphere (greater than or equal to 0 degrees latitude) and Southern Hemisphere (less than 0 degrees latitude):

* Northern Hemisphere - Temperature (F) vs. Latitude
* Southern Hemisphere - Temperature (F) vs. Latitude
* Northern Hemisphere - Humidity (%) vs. Latitude
* Southern Hemisphere - Humidity (%) vs. Latitude
* Northern Hemisphere - Cloudiness (%) vs. Latitude
* Southern Hemisphere - Cloudiness (%) vs. Latitude
* Northern Hemisphere - Wind Speed (mph) vs. Latitude
* Southern Hemisphere - Wind Speed (mph) vs. Latitude

#After each pair of plots explain what the linear regression is modeling such as any relationships you notice and any other analysis you may have

In [ ]:
#After each pair of plots explain what the linear regression is modeling such as any relationships you notice and any other analysis you may have